<a href="https://colab.research.google.com/github/bltsezer/DataSet_Uuygulama/blob/main/_05_BelignMalignTestCase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Veri indirme ve düzenleme

Verisetimiz, derideki kanser hücrenin iyi huylu veya kotu huylu olduğunu tahmin eden bir örnek bir test uygulamasıdır.


Veriler Kaggle platformdan Google Drive ortamına eklenmiştir.
Buradaki komut ile kaggledeki dataset Drive yüklenirVeriler Kaggle platformdan Google Drive ortamına eklenmiştir.(Bulunduğu klasöre dikkat edin)

In [ ]:
#pwd
!kaggle datasets download -d fanconic/skin-cancer-malignant-vs-benign


Verisetinin bulunduğu klasöre gitmek için os modulu import edilip aut yetkisinden sonra işlemi başlatıyoruz
görüntü veri setinin bulundugu klasör git ve gerekli os ayarlamaları yap

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
import os
os.environ["KAGGLE_CONFIG_DIR"]="/content/gdrive/MyDrive/Kaggle"

Tüm dosya/dizin işlemleri collab ortamımda düzenlenmistir.
Dataseti hangi klasorunuzdeyse orada calistiriniz.

In [ ]:
#%cd "/content/gdrive/MyDrive/
#driver icerisinde Kaggle klasoru olusturuldu
#mkdir Kaggle
#os.environ["KAGGLE_CONFIG_DIR"]+="/Kaggle"
#%cd "/content/gdrive/MyDrive/Kaggle"

zip dosyanından cıkartalim

In [ ]:
!unzip benign.zip
!unzip malignant.zip
#listele
!ls
#bulunduğu dizin
!pwd
#zip dosyayını silmek ismiyorsanız son iki komutu yazmayabiliriz
#ben yer işgal etmesin diye siliyorum 
!rm benign.zip
!rm malignant.zip

Kullanacağımız tüm paketleri import ediyoruz.İşlemleri yapacak dizine gidiyoruz

In [ ]:
import numpy as np
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
from PIL import Image
%cd "/content/gdrive/MyDrive/Kaggle"

VERİ SETİNİ LİNK ÜZERİNDEN İNDİRME

#link üzeinden dosya indirme


Görüntünün bulunduğu dizin ve ve dosya içerisinde bulunan görüntü sayısını bir değere atayalım
 Test ve train verilerini ayıralım

In [ ]:
#dizini ve dosya yolu
file_path="/content/gdrive/MyDrive/Kaggle/data"
data_dir = pathlib.Path(file_path)
#kontrol için tüm görüntü saısı
imageCount=len(list(data_dir.glob("*/*/*.jpg")))
#verileri manuel olarak path ataması
TRAIN_bening    = file_path+str("/train/benign")
TRAIN_malignant = file_path+str("/train/malignant")
TEST_bening     = file_path+str("/test/benign")
TEST_malignant  = file_path+str("/test/malignant")
#verileri görüntü listesi olaralk list ataması
TRAIN_bening_list   = list(data_dir.glob("train/benign/*.jpg"))
TRAIN_malignant_list= list(data_dir.glob("train/malignant/*.jpg"))
TEST_bening_list    = list(data_dir.glob("test/benign/*.jpg"))
TEST_malignant_list = list(data_dir.glob("test/malignant/*.jpg"))


görüntü dataseti yükleyip, matrise dönüstürelim.

In [ ]:
joinLambda =lambda image_name : np.asarray(Image.open(image_name).convert("RGB"))
print("...")
#egitim verisi klasorundeki herbir görüntünün iyi huylu görüntüsünü sec ve liste ekle
IMG_train_benign = [joinLambda(os.path.join(TRAIN_bening, filename)) for filename in os.listdir(TRAIN_bening)]
#matrise dönustur
X_BENIGN = np.array(IMG_train_benign, dtype='uint8')
#egitim verisindeki herbir görüntünün kotu huylu görüntüsünü sec ve liste ekle
IMG_train_malignant = [joinLambda(os.path.join(TRAIN_malignant, filename)) for filename in os.listdir(TRAIN_malignant)]
#matrise dönustur
X_MALIGNANT = np.array(IMG_train_malignant, dtype='uint8')
print("...")

In [ ]:
#test verisi klasoründeki herbir görüntünün iyi huylu görüntüsünü sec ve liste ekle
IMG_test_benign = [joinLambda(os.path.join(TEST_bening, filename)) for filename in os.listdir(TEST_bening)]
X_BENIGN_TEST = np.array(IMG_test_benign, dtype='uint8')
#test verisi klasoründeki herbir görüntünün kotu huylu görüntüsünü sec ve liste ekle
IMG_test_malignant = [joinLambda(os.path.join(TEST_malignant, filename)) for filename in os.listdir(TEST_malignant)]
#matrise donustur
X_MALIGNANT_TEST = np.array(IMG_test_malignant, dtype='uint8')
print("...")

Sınıflandırma için etiketleri tanımlayalım.
zeros:Kotu huylu degil
ones:Kotu Huylu

In [ ]:
#Kotu huylu==0 ise iyi huyludur
#kotu huyle==1 ise kotu huyludur
np.zeros(X_BENIGN.shape[0])
np.ones(X_MALIGNANT.shape[0])
np.zeros(X_BENIGN_TEST.shape[0])
ones(X_MALIGNANT_TEST.shape[0])

In [ ]:
Y_Benign    = np.zeros(X_BENIGN.shape[0])
Y_Malignant = np.ones(X_MALIGNANT.shape[0])

Y_TEST_Benign    = np.zeros(X_BENIGN_TEST.shape[0])
Y_TEST_Malignant = np.ones(X_MALIGNANT_TEST.shape[0])

2.AYRI VERİ SETLERİ OLUSTURALIM 

Egitim için iyi huylu ve kotu huylu DATA birleştir

Egitim için iyi huylu ve kotu huylu LABEL  birleştir

TEST için iyi huylu ve kotu huylu DATA birleştir

TEST için iyi huylu ve kotu huylu LABEL  birleştir


In [ ]:
X_TRAIN = np.concatenate((X_BENIGN,X_MALIGNANT),axis=0)
Y_TRAIN = np.concatenate((Y_Benign,Y_Malignant),axis=0)


X_TEST = np.concatenate((X_BENIGN_TEST,X_MALIGNANT_TEST),axis=0)
Y_TEST = np.concatenate((Y_TEST_Benign,Y_TEST_Malignant),axis=0)

Verileri karıştır.shuffling)

In [ ]:
len_train = np.arange(X_TRAIN.shape[0])
np.random.shuffle(len_train)
X_TRAIN=X_TRAIN[len_train]
Y_TRAIN=Y_TRAIN[len_train]

len_train = np.arange(X_TEST.shape[0])
np.random.shuffle(len_train)
X_TEST =X_TEST[len_train]
Y_TEST =Y_TEST[len_train]

ön aşama için ayrılmış veri setlerinin boyutlarını inceleyelim

In [ ]:
print("egitim verisi boyut:",X_TRAIN.shape)
print("test verisi boyutu:",X_TEST.shape)
print("Goruntu sayısı:",len(X_TRAIN))
print("")
print("Egitimdeki etiket boyutu:",Y_TRAIN.shape)
print("Testdeki etiket boyutu:",Y_TEST.shape)
print("Etiket Sayısı:",len(Y_TRAIN))

In [ ]:
class_names=["Iyi","kotu"]

örnek 25 tane esmi görselleştirelim

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for i in range(25):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(X_TRAIN[i], cmap=plt.cm.binary)
  if Y_TRAIN[i]==1:
    plt.xlabel(class_names[int(Y_TRAIN[i])]+" huylu")
  else:
    plt.xlabel(class_names[int(Y_TRAIN[i])]+" huylu")
plt.show()


bar grafi üzerinde inceleyelim

In [ ]:
plt.bar(0, Y_TRAIN[np.where(Y_TRAIN == 0)].shape[0], label = 'iyi huylu')
plt.bar(1, Y_TRAIN[np.where(Y_TRAIN == 1)].shape[0], label = 'kotu huylu')
plt.legend()
plt.title("Training Data")
plt.show()

plt.bar(0, Y_TEST[np.where(Y_TEST == 0)].shape[0], label = 'benign')
plt.bar(1, Y_TEST[np.where(Y_TEST == 1)].shape[0], label = 'malignant')
plt.legend()
plt.title("Test Data")
plt.show()

##Model

In [ ]:
#height,width,channel degerlerini tek tek secelim
H,W,C=X_TRAIN.shape[1],X_TRAIN.shape[2],X_TRAIN.shape[3]
print("h: ",H)
print("w: ",W)
print("c: ",C)
#
#********Model olusturma
#giris resim matrisindeki tümdegerler giris katmanında
#128 ve 64 adet nöron 2 gizli katman eklenecek ve RELU ile aktivite(RELU enuygun aktivasyon secildi) edilecek
#cıkış katmanı 2 nörondan olusacak(B/M)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(H, W,C)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

modelin optimizer,loss(kayıp),ve metrik degerlerini belirtip compile edilecek.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['categorical_accuracy'])

modeli eğitelim(fitting)

In [ ]:
model.fit(X_TRAIN, Y_TRAIN, epochs=5)

kayıp de dogruluk degerlerini ekrana basalim

In [ ]:
test_loss, test_acc = model.evaluate(X_TEST,  Y_TEST, verbose=2)

print('\nTest accuracy:', test_acc)

Test verisi üzerinden tahminleyici(prediction) olusturalım

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(X_TEST)
#numune olarak 5 tanesini yazalim
for i in range(5):
  a=np.random.randint(20)
  print(predictions[a])
  print("\n")

##Verileri görselleştirelim

görüntüyü ve tahmini skorları çizecek fonksiyon yazalim

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("Gercek:{} {:2.0f}% ".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(2))
  plt.yticks([])
  thisplot = plt.bar(range(2), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)
  

  thisplot[predicted_label].set_color('red')
  thisplot[int(true_label)].set_color('blue')
  plt.xlabel("Tahmin:({})".format(
                                class_names[int(true_label)],
                                color="c")
  )

#12 rastgele görüntüyü tahmin edelim 
num_rows = 6
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], Y_TEST, X_TEST)

  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], Y_TEST)
plt.tight_layout()
plt.show()